In [22]:
import os
import pandas as pd
import snscrape.modules.twitter as sntwitter
import numpy as np
import datetime
from capstone_twitter_search import twittsearch
import os
import pandas as pd

import numpy as np                               # linear algebra
import pandas as pd                              # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re                                        # to handle regular expressions
from string import punctuation                   # to extract the puntuation symbols
import nltk
from nltk.tokenize import word_tokenize          # to divide strings into tokens
from nltk.stem import WordNetLemmatizer          # to lemmatize the tokens
from nltk.corpus import stopwords                # to remove the stopwords 

import random                                    # for generating (pseudo-)random numbers
import matplotlib.pyplot as plt                  # to plot some visualizations
import datetime
import tensorflow as tf            
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding
from tensorflow.keras.callbacks import EarlyStopping
from mordecai import Geoparser
#from mordecai import batch_geoparse
from flatten_json import flatten_json
import numpy as np
import json
from tensorflow.keras import layers
import text_proccessing
from text_proccessing import clean_text
from text_proccessing import remove_stopwords
from text_proccessing import lemmatize_text
from text_proccessing import concatenate_text
from text_proccessing import makeglove
from text_proccessing import make_embedding_matrix
import append_df
from append_df import append_df
import modhelp
from modhelp import train_val_split
from modhelp import geo_df
from modhelp import suggest_nn2

In [8]:
code_dir = os.getcwd()
directory = os.path.join(data_dir,'splits')
parent_dir = os.path.dirname(code_dir)
data_dir = os.path.join(parent_dir,"Data")
dsa= os.path.dirname(parent_dir)
tweet_dir = os.path.join(parent_dir,"TweetMap")


In [14]:
since_date = '2021-07-13'
until_date = '2021-07-14'

In [6]:
train_data = pd.read_csv(os.path.join(directory, 'InformativenessTrain_Tokenized.csv'),engine='python')
train_data['text']= train_data['text'].astype(str)

In [9]:
start =  datetime.datetime.now() 
print('splitthis started at: {}'.format(start))
train_samples, val_samples, train_labels, val_labels = train_val_split(train_data, 0.25)
end =  datetime.datetime.now() 
print ('Text done {}'.format(end-start))
start =  datetime.datetime.now() 
print('Loading model,embedding and glove commenced at  {}'.format(start))

model = tf.keras.models.load_model(os.path.join(dsa,'model1'))
print("loading vecotor")
path_to_glove_file = os.path.join(dsa,'WordVector','glove.twitter.27B.200d.txt')

print("indexing")
embeddings_index=makeglove(path_to_glove_file)
print("matrix and vectorize")
embedding_matrix, vectorizer = make_embedding_matrix(train_samples, val_samples, embeddings_index)
end =  datetime.datetime.now() 

print("embedded in {}".format(end-start))

splitthis started at: 2021-07-15 17:06:24.118237
Total size of the dataset: 117339.
Training dataset: 88005.
Validation dataset: 29334.
Text done 0:00:00.304603
Loading model,embedding and glove commenced at  2021-07-15 17:06:24.423653
loading vecotor
indexing
matrix and vectorize
Converted 32239 words (22761 misses).
embedded in 0:03:51.935787


In [15]:
twts = pd.read_csv(os.path.join(tweet_dir,'tweets_df_{0}_{1}.csv'.
                              format(since_date.replace('-',''),until_date.replace('-',''))), engine = 'python')
twts['Text'] = twts['Text'].astype(str)


In [23]:
procstart =  datetime.datetime.now()
print("{}: processing ".format(procstart))
twts['ptext'] = twts['Text'].apply(lambda x: clean_text(x))
twts['ptext'] = twts['ptext'].apply(lambda x: word_tokenize(x))
twts['ptext'] = twts['ptext'].apply(lambda x : remove_stopwords(x))
twts['ptext'] = twts['ptext'].apply(lambda x : lemmatize_text(x))
twts['ptext'] = twts['ptext'].apply(lambda x : concatenate_text(x))
procend=  datetime.datetime.now() 
print("{}: processed in {}".format(procend, procend - procstart))



2021-07-15 17:17:12.837638: processing 
2021-07-15 17:20:43.449239: processed in 0:03:30.611601


In [24]:

predstart = datetime.datetime.now() 
print("{}: predict".format(predstart))
predictions = suggest_nn2(twts, model,vectorizer)

submission_data = {"ID": twts['TweetId'].tolist(),"tweet": twts['Text'].tolist(), "target": predictions}

submission_df = pd.DataFrame(submission_data)
result = twts.join(submission_df.target)
predend = datetime.datetime.now() 
print("predicted {}".format(predend-predstart))
# 
result_inf = result[result['target']==0]
print("working over {} informative tweets".format(len(result_inf)))



2021-07-15 17:23:55.253515: predict
predicted 0:01:30.184650
working over 40821 informative tweets


In [25]:


print("Run geoprocessing over tweets")

geostart =  datetime.datetime.now() 
print("{}: geostart".format(geostart))
df_js = geo_df(result_inf)
geoend =  datetime.datetime.now()
print("{}: geoend {}".format(geoend,geoend-geostart))

df_js.to_csv(os.path.join(tweet_dir,'result_{0}_{1}.csv'.
                              format(since_date.replace('-',''),until_date.replace('-',''))))
print("fín")

Run geoprocessing over tweets
2021-07-15 17:29:21.311779: geostart
Models path: /Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/mordecai/models/
You may be using an outdated Geonames index/Mordecai version.
Your index is from 2018-06-05, while your Mordecai version is from 2020-07-11. Please see
https://github.com/openeventdata/mordecai/ for instructions on updating.


  0%|          | 126/40821 [00:15<2:13:43,  5.07it/s]Traceback (most recent call last):
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/mordecai/geoparse.py", line 726, in infer_country
    prediction = self.country_model.predict(i['matrix']).transpose()[0]
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 1629, in predict
    tmp_batch_outputs = self.predict_function(iterator)
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 862, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning

None
None


  1%|          | 252/40821 [00:25<46:12, 14.63it/s]  Traceback (most recent call last):
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/mordecai/geoparse.py", line 726, in infer_country
    prediction = self.country_model.predict(i['matrix']).transpose()[0]
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 1629, in predict
    tmp_batch_outputs = self.predict_function(iterator)
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 862, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning

None


...

None


100%|█████████▉| 40808/40821 [46:18<00:01, 11.88it/s]Traceback (most recent call last):
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/mordecai/geoparse.py", line 726, in infer_country
    prediction = self.country_model.predict(i['matrix']).transpose()[0]
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 1629, in predict
    tmp_batch_outputs = self.predict_function(iterator)
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py", line 862, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/Volumes/Elements/DataScience/anaconda3/envs/machinelearning

None


100%|██████████| 40821/40821 [46:19<00:00, 14.68it/s]
/Volumes/Elements/DataScience/dsa/capstone/Code/modhelp.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['TweetId'][i]=tweets_df['TweetId'][row]


KeyError: 45